In [ ]:
!pip install git+https://github.com/huggingface/transformers SentencePiece
!git clone https://github.com/huggingface/transformers
!pip install -r /content/transformers/examples/pytorch/summarization/requirements.txt

Стандартная модель rut5 обучается через скрипт суммаризатора бибилиотеки трансформер

In [ ]:
!python /content/transformers/examples/pytorch/summarization/run_summarization.py \
    --seed 42 \
    --model_name_or_path "sberbank-ai/ruT5-large" \
    --num_train_epochs 3 \
    --warmup_ratio 0.1 \
    --do_train \
    --do_eval \
    --train_file /content/train.csv \
    --validation_file /content/valid.csv \
    --source_prefix "" \
    --output_dir /content/rut5-large_1/ \
    --overwrite_output_dir \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --text_column 'toxic_comment' \
    --summary_column 'neutral_comment' \
    --save_total_limit 8 \
    --gradient_accumulation_steps 1 \
    --logging_strategy "epoch" \
    --evaluation_strategy "epoch" \
    --save_strategy "epoch" \
    --predict_with_generate \
    --max_source_length 128 \
    --max_target_length 128 \

Модели DisCO И LevT Обучаются через измененную бибилотеку fairseq

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-11.1
git clone https://github.com/jungokasai/deep-shallow.git
cd deep-shallow
python setup.py build_ext --inplace
sudo -E pip install --editable ./

In [ ]:
# ЕСЛИ ЧТО-ТО СЛОМАЛОСЬ
! echo $PYTHONPATH
import os 
os.environ['PYTHONPATH'] += ":/content/deep-shallow/"
! echo $PYTHONPATH

Для дистилляция датасета используется русский корпус Лейпцига

In [ ]:
#ЭТО ДИСКО
fairseq-train \
    /paraphrase_ru_big_bin/ \
    --save-dir /checkpoints \
    --ddp-backend=no_c10d \
    --task translation_lev \
    --criterion nat_loss \
    --arch disco_transformer \
    --noise no_noise  --fp16 \
    --share-all-embeddings \
    --optimizer adam --adam-betas '(0.9,0.98)' \
    --lr 0.0005 --lr-scheduler inverse_sqrt \
    --min-lr '1e-09' --warmup-updates 10000 \
    --warmup-init-lr '1e-07' --label-smoothing 0.1 \
    --dropout 0.2 --weight-decay 0.01 \
    --decoder-learned-pos \
    --encoder-learned-pos \
    --apply-bert-init \
    --log-format 'simple' --log-interval 1000 \
    --fixed-validation-seed 7 \
    --max-tokens 16000 \
    --save-interval-updates 10000 \
    --max-update 300000 --skip-invalid-size-inputs-valid-test

# ЭТО ЛЕВЕНШТЕЙН
fairseq-train \
    /paraphrase_ru_big_bin/ \
    --save-dir /checkpoints \
    --ddp-backend=no_c10d \
    --task translation_lev \
    --criterion nat_loss \
    --arch levenshtein_transformer \
    --noise random_delete \
    --share-all-embeddings \
    --optimizer adam --adam-betas '(0.9,0.98)' \
    --lr 0.0005 --lr-scheduler inverse_sqrt \
    --min-lr '1e-09' --warmup-updates 10000 \
    --warmup-init-lr '1e-07' --label-smoothing 0.1 \
    --dropout 0.3 --weight-decay 0.01 \
    --decoder-learned-pos \
    --encoder-learned-pos \
    --apply-bert-init \
    --log-format 'simple' --log-interval 1000 \
    --fixed-validation-seed 7 \
    --max-tokens 16000 \
    --save-interval-updates 10000 \
    --max-update 300000 --skip-invalid-size-inputs-valid-test

In [ ]:
#ГЕНЕРАЦИЯ ДИСКО
#                        [--iter-decode-eos-penalty N]
#                        [--iter-decode-max-iter N]
#                        [--iter-decode-force-max-iter]
#                        [--iter-decode-with-beam N]
#                        [--iter-decode-with-external-reranker]
!fairseq-generate \
    /detox_ru_bin \
    --gen-subset predictnew10 --dataset-impl raw \
    --task translation_lev \
    --path /content/russe_detox_2022-main/checkpoint_last.pt \
    --iter-decode-max-iter 12 \
    --iter-decode-eos-penalty 0 \
    --beam 8 --remove-bpe \
    --print-step \
    --batch-size 220 \
    --decoding-format easy-first --skip-invalid-size-inputs-valid-test  > /content/tmp/data_text40

In [ ]:
#ГЕНЕРАЦИЯ ЛЕВЕНШТЕЙН
#    --path /new_models_disco_and_lev/checkpoint_best_lev.pt \
!fairseq-generate \
    /detox_ru_bin \
    --gen-subset test_small --dataset-impl raw \
    --task translation_lev \
    --path /new_models_for_detox/checkpoint_best2_lev.pt \
    --iter-decode-max-iter 16 \
    --iter-decode-eos-penalty 0 \
    --beam 6 --remove-bpe \
    --print-step \
    --batch-size 100 \
    > /content/tmp/data_text10

In [ ]:
!mkdir /content/natural_order

!grep ^S /content/tmp/data_text10 | LC_ALL=C sort -V | cut -f2- > /content/natural_order/src.txt
!grep ^T /content/tmp/data_text10 | LC_ALL=C sort -V | cut -f2- > /content/natural_order/ref.txt
!grep ^H /content/tmp/data_text10 | LC_ALL=C sort -V | cut -f3- > /content/natural_order/hyp.txt